<a href="https://colab.research.google.com/github/phalves/ConFIA/blob/main/Sandboxing_Open_Banking_Scenario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring Simulation Scenarios to Mitigate Information Asymmetry Under the LGPD Perspective - Open Banking Scenario

### Scenario Description: Open Banking

Open banking is a practice of enabling data sharing between financial institutions. This practice allowed banking interoperability by APIs (Application Programming Interfaces). For example, the data subject can request a credit card from bank A, a financial loan from bank B, and buy assets from bank C. Moreover, open banking allows, for instance, the data subject to open an account just requesting his/her data from an institution that he/she has an account previously. Therefore, open banking turns the data sharing more agile, transparent, and secure for providing resources to the data subject chose: (i) which data he/she wants to share; (ii) when; (iii) how long, and (iv) with whom he/she wants to share.

According to indicators presented by [Open Banking Implementation Entity (OBIE)](https://www.openbanking.org.uk/api-performance/), since 2020, the number of successful API calls per month raised from 400.000 in May to more than 1.000.000 in 2022. It corresponds to an increase of 250% in two years. Moreover, the number of successful payments per month raised from 280 in July 2020 to more than 520.000 in May 2022.
They are some indicators that can demonstrate the evolution of open banking in the UK.

Thus, this sandbox seeks to provide an experimentation environment to the actors involved, i.e., Data Subjects, Controllers, and Processors. This environment was developed in Python based on the documentation supplied by [OBIE](https://www.openbanking.org.uk/about-us/). The OBIE creates software standards and industry guidelines to drive competition, innovation and transparency in UK retail banking.


To know more about OpenBanking, please access LINK

 ---------

### Creating Actors
In order to prepare the environment for exploring a data regulation, first we need to create the actors that will perform actions in our sandbox.

#### Data Subject
The first actor is the Data Subject. For this actor, we will will define the following data structure:
- Data Subject ID, i.e., an identification number
- Data Subject Consent List, i.e., a list of consents available for the Data Subject



In [1]:
class DataSubject:
  def __init__(self,id):
    self.id = id                 # Identification Number
    self.consents = []           # Empty list of consents
    self.personalData = {}       # Dictionary of Personal Data
    self.transactionalData = {}  # Dictionary of Transactional Data

Now, we can create the person John. His ID is 35512367800.

In [2]:
john = DataSubject('35512367800')
john.personalData = {'name':'John','email':'john@mail.com','cellphone':'+1 98638293756'}

You can verify if John was created correctly by printing his data

In [3]:
print('ID:')
print(john.id)

print('\n')

print('Consents:')
print(len(john.consents))

ID:
35512367800


Consents:
0


As expected, there is no consent associated to John for a while.

#### Data Controller
For this actor, we will will define the following data structure:
- Data Controller ID, i.e., an identification number
- Data Controller Consent List, i.e., a list of consents issued by the Data Controller
- Data Controller Contact, i.e., an address or other information that allow the Data Subject to send questions and requests.
- Data Controller Communication Channel, i.e., the means that the Data Subject can contact the Data Controller.
- Dictionary where the dict key represents the Data Subject ID and the dict value represents a boolean (**true** or **false**) to indicate if the Data Controller is processing or not information from the Data Subject. If a dict key presents **no value**, it means the Data Controller never processed the Data Subject information. If a dict key presents a **'false'** value, it means that the Data Controller processed the information in the past, but now it is not processing anymore. Last but not least, if a dict key presents the a **'true'** value, it means that the Data Controller is processing the Data Subject information.

In [4]:
class DataController:
  def __init__(self,id,contact,communiactionChannel):
    self.id = id
    self.consents = []
    self.contact = contact
    self.communiactionChannel = communiactionChannel
    self.isProcessing = {} #Dictionary DataSubjectID:Boolean

Now, we can create the company Bank A. Its ID is 00345 and the Data Subjects can contact the Data Controller by mail (contact@banka.com).

In [5]:
bankA = DataController('00345','contact@banka.com','email')

Bank details:

In [6]:
print('ID:')
print(bankA.id)

print('\n')

print('Consents:')
print(len(bankA.consents))

print('\n')

print('Contact:')
print(bankA.contact)

print('\n')

print('Communication Channel:')
print(bankA.communiactionChannel)

ID:
00345


Consents:
0


Contact:
contact@banka.com


Communication Channel:
email


In order to create the required actors in this scenario, follow below the code to instanciate Bank B. John will share his data from Bank A to Bank B in the next scenes.

In [7]:
bankB = DataController('48634','contact@bankb.com','email')

Moreover, in order to provide a history of actions, the following code provides the structure to generate an action log based on the Consent Ontology.

In [8]:
# These libraries are used to manipulate dates 
from datetime import date, datetime 

# Action Types
Communication = 'Communication'
Explanation = 'Explanation'
Deletion = 'Deletion'

# Deontic Concepts
Obligation = 'Obligation'
Permission = 'Permission'
Prohibition = 'Prohibition'
Violation = 'Violation'
Compliance = 'Compliance'

class Log:
  def __init__(self):
    #empty log
    self.root = ['Date; Event; Type; Deontic Concept']
    self.filePersistence = False
    

  def configLogPersistence(self,_fileName):
    if _fileName!='':
      self.filePersistence = True
      
      #open file
      self.file = open(_fileName, "w")

    else:
      print('There is no fileName, so the file will not be generated')
  
  def registerAction(self,message,actionType,deonticConcept):
    self.date = datetime.today()
    self.message = message
    self.actionType = actionType
    self.deonticConcept = deonticConcept
    self.root.append(str(self.date)+'; '+str(self.message)+'; '+str(self.actionType)+';  '+str(self.deonticConcept))

    if self.filePersistence==True:
      self.file.write(self.root[-1]) # Save the most recent action

  def printLog(self):
    for action in self.root:
      print(action)


The code below creates an instance of the log class, and it will be used in the next scenes to persist the log of actions performed by the actors.

In [9]:
log = Log()

---------

#### Scene 1 - Setting up the consent term
A data subject which aims to share its financial data should be aware which permissions it is giving to the data controller.

The [Open Banking Consent API Documentation](https://openbanking.atlassian.net/wiki/spaces/DZ/pages/1077805332/Account+Access+Consents+v3.1.2) presents the following UML Diagram, which depicts the attributes required when the data subject is requested to share its data.

![Consent Attributes](https://openbanking.atlassian.net/wiki/download/attachments/1077805332/OBReadConsentResponse1.gif?version=1&modificationDate=1556635325535&cacheVersion=1&api=v2)

This means that there are many permissions that a data controller may request:
ReadAccountsBasic,
ReadAccountsDetail,
ReadBalances,
ReadBeneficiariesBasic,
ReadBeneficiariesDetail,
ReadDirectDebits,
ReadOffers,
ReadPAN,
ReadParty,
ReadPartyPSU,
ReadProducts,
ReadScheduledPaymentsBasic,
ReadScheduledPaymentsDetail,
ReadStandingOrdersBasic,
ReadStandingOrdersDetail,
ReadStatementsBasic,
ReadStatementsDetail,
ReadTransactionsBasic,
ReadTransactionsCredits,
ReadTransactionsDebits,
ReadTransactionsDetail.

For our context, let's consider only these four: ReadAccountsBasic ReadAccountsDetail, ReadBalances, ReadOffers, and ReadProducts.

However, there are more attributes that we should list in order to compy with Data Regulation. 

For Data Regulation, we considered the LGPD (Brazilian Data Protection Law) and the [Consent Ontology](https://github.com/phalves/ConFIA/blob/main/ConsentOntology.md), but you can change the attributes by executing an ontology alignment. This alignment will enable you to reflect the ontology attributes on this sandbox.

----

The first step is the consent setup. The consent must have all information described in the LGPD Art. 9. The following method receives all the required information.

**General information (Data Subject, Data Controller, Personal and Sensitive Data)**

The **Data Subject John** allows the **Data Controller Bank B** to access, store, and process his personal and transactional data from Bank A in order to **offer products and services**, **allowing Bank B to send offers appropriately based on John's data**. 

**Data Controller collection, processing, and storing guidelines**

John's personal and transactional data will be shared with Bank B under strict governance policies that guarantee that only the information required to execute their functions will be shared. The employees will respond to any unauthorized data access, leak, or other activities that may expose or cause any loss to the data subject. None information will be publicly available without a previous consent acceptance.

**Processing and storing time**

The personal and transactional data will be available, stored, and processed while the data subject has an active consent term with Bank A and Bank B. **A new consent term will be required in two situations:**
 - when there is an update on the consent term;
 - when the data subject changes the data that he wants to share or change the time range;
 - when the due date is accomplished.

**Consent expiration date**
Therefore, this **consent term is valid for twelve months at most considering the open banking rules**. The data subject may renew the consent or revoke it at any time.

**Security measures**
Bank B will apply **cryptographic algorithms** and **access policies** to avoid data breaches and unauthorized access. The personal and transactional data will be **stored in a private cloud** where Bank B has complete control of applied technologies. 

**Third-party data sharing**
The Data Controller Bank B is not allowed to **share** the Data Subject data, except for cases that the government requires such data.

**Contact information**
To make any request, please use the Data Controller communication channel by **email lgpd@bankb.br**.


PS: The Data Controllers must inform in any case if they are processing the personal data if requested.

---

The following code aims to represent the consent term, its status and the Data Subject and the Data Controller's Rights. Questions regarding these Rights emerge when a concept is accepted or revoked.

In [10]:
## Libraries ##
# This library is used to generate a unique identification number
import uuid 

# This library is used to perform adding numbers to months to walk back and forward from the current time
from dateutil.relativedelta import relativedelta 

## Global Static Variables ###
# Consent Status possibilities
AwaitingAuthorisation = 'AwaitingAuthorisation'
Authorised            = 'Authorised'
Rejected              = 'Rejected'
Revoked               = 'Revoked'

# Data Subject Rights
DataAccess        = 'Data Access'
DataCopy          = 'Data Copy'
DataCorrection    = 'Data Correction'
DataAnonymization = 'Data Anonymization'
DataPortability   = 'Data Portability'
DataDeletion      = 'Data Deletion'
ThirdPartySharing = 'Information regarding the data sharing with a third party'
ConsentRevocation = 'Request consent revocation'

# Data Controller Rights
DataProcessing  = 'Data Processing'
DataCollecting  = 'Data Collecting'
DataStoring     = 'Data Storing'

# Consent Class
class Consent:

  # Create Consent Function based on the required information described in the current Data Regulation
  # This function expects to receive the consent purpose, the specific purpose,
  #  the third party sharing purpose (if needed), the data controller information,
  #  how long this consent is valid, and how many months before the creation data 
  #  the Data Controller can get the Data Controller data.
  def createConsent(self, purpose, specificPurpose, thirdParySharingPurpose, criptographyAlgoritm, accessPolicies, storagePlatform, dataController, validForQuantityOfMonths, monthsBeforeTheAcceptance):
    
    # Open Banking API Params
    self.id = uuid.uuid4()                   # Unique Consent ID is generated when the function is called
    self.creationTime = datetime.today()     # Creation Time is generated when the function is called
    self.status = AwaitingAuthorisation 
    self.permissions = ['ReadAccountsBasic','ReadAccountsDetail','ReadBalances','ReadOffers','ReadProducts']
    self.expirationDateTime = self.creationTime + relativedelta(months=validForQuantityOfMonths)
    self.transactionFromDateTime = self.creationTime - relativedelta(months=monthsBeforeTheAcceptance)
    self.transactionToDateTime = self.expirationDateTime
    self.statusUpdatedTime = datetime.today()


    # Data Regulation Params
    # Here you can add the params required for a consent ruled by the data regulation
    self.purpose = purpose
    self.specificPurpose = specificPurpose
    self.thirdParySharingPurpose = thirdParySharingPurpose
    self.dataController = dataController
    self.dataSubjectRights = []
    self.dataControllerRights = []
    self.criptographyAlgoritm = criptographyAlgoritm
    self.accessPolicies = accessPolicies
    self.storagePlatform = storagePlatform

    # Log
    log.registerAction('A new consent term was generated, but it was not accepted yet ID: '+str(self.id)+' Current Status: '+str(self.status),Communication,Permission)

  # Accept Concent Function
  # This function aims to change the consent status (folloing the Open Baking API guidelines),
  #  update the Data Subject and the Data Controller Rights (following the Data Regulation guidelines)
  def acceptConsent(self,dataSubject,dataController):
    # Consent atrributes
    self.status = Authorised
    self.startDateTime = datetime.today()
    self.statusUpdatedTime = datetime.today()

    # Data that will be shared
    if len(dataSubject.personalData)>0:
      self.personalData = dataSubject.personalData
    
    if len(dataSubject.transactionalData)>0:
      self.transactionalData = dataSubject.transactionalData

    # Data Subject and Data Controller Rights
    self.dataSubjectRights = [DataAccess,DataCopy,DataCorrection,
                              DataAnonymization,DataPortability,
                              DataDeletion,ThirdPartySharing,
                              ConsentRevocation]
    self.dataControllerRights = [DataProcessing,
                                 DataCollecting,
                                 DataStoring]
    dataController.isProcessing[dataSubject.id]=True

    log.registerAction('Data Subject '+ dataSubject.id +' accepted the consent term',Communication,Permission)
    log.registerAction('Data Subject can now have all foressen rights',Explanation,Permission)
    log.registerAction('Data Controller '+ dataController.id +' can process the Data Subject information from '+str(self.startDateTime.date())+' to '+str(self.expirationDateTime.date()),Explanation,Permission)

  # Revoke Concent Function
  # This function aims to change the consent status (folloing the Open Baking API guidelines),
  #  update the Data Subject and the Data Controller Rights (following the Data Regulation guidelines)
  def revokeConsent(self,dataSubject,dataController):
    if ConsentRevocation not in self.dataSubjectRights:
      log.registerAction('Data Controller '+ dataController.id +' has not an available consent from '+dataSubject.id,Explanation,Permission)
      return False
    
    # Consent atrributes
    self.status = Revoked
    self.expirationDateTime = datetime.today()
    self.statusUpdatedTime = datetime.today()
    
    # Data Subject and Data Controller Rights
    self.dataSubjectRights = [DataAccess,DataCopy,DataCorrection,
                              DataAnonymization,DataPortability,
                              DataDeletion]
    self.dataControllerRights = [DataStoring]
    dataController.isProcessing[dataSubject.id]=False

    log.registerAction('Data Subject '+ dataSubject.id +'  requested to the Data Controller to revoke his/her consent',Communication,Prohibition)
    log.registerAction('From now, the Data Controller '+ dataController.id +' cannot collect the Data Subject information',Communication,Prohibition)
    log.registerAction('From now, the Data Controller '+ dataController.id +' cannot process the Data Subject information',Communication,Prohibition)
    log.registerAction('From now '+str(self.expirationDateTime.date())+', consent '+ str(self.id) +' is not valid to be used by the data controller',Explanation,Prohibition)

    return True

Now, lets create a new consent following the above scenario description.

In [11]:
consentBankA = Consent()
consentBankA.createConsent('offer_products_and_services','create_specific_offers',
                           'anonymized_data_analytics','SHA256',
                           'Authorized employees can access the data only',
                           'Bank A private cloud',
                           bankA,60,1)

And lets print some data from this consent term.

In [12]:
print('---- A new consent was create ----\n')
print('Consent unique identification code:  '+str(consentBankA.id))
print('This consent was created on:         '+str(consentBankA.creationTime.date()))
print('Current consent status:              '+str(consentBankA.status))
print('The Data Controller will have the following permissions: '+str(consentBankA.permissions))
print('This consent will expire on:         '+str(consentBankA.expirationDateTime.date()))
print('The data controller will be able to access the data subject transactions from: '+str(consentBankA.transactionFromDateTime.date())+' to '+str(consentBankA.transactionToDateTime.date()))

---- A new consent was create ----

Consent unique identification code:  c0a0ee7c-cffc-4a02-aafb-ba9bd228d8f5
This consent was created on:         2022-08-08
Current consent status:              AwaitingAuthorisation
The Data Controller will have the following permissions: ['ReadAccountsBasic', 'ReadAccountsDetail', 'ReadBalances', 'ReadOffers', 'ReadProducts']
This consent will expire on:         2027-08-08
The data controller will be able to access the data subject transactions from: 2022-07-08 to 2027-08-08


Now, let's insert the John's transactional data

In [13]:
john.transactionalData = {'Account Balance':250,
                           'Credit Card Type':'Platinum',
                           'Limit':5000, 
                           'Transactions':{'supermarket':300,
                                           'library':700,
                                           'drugstore':100}}

And lets associate the consent term to John

In [14]:
john.consents.append(consentBankA)

log.registerAction('Consent requested to the Data Subject. ID: '+str(consentBankA.id)+' Current Status: '+str(consentBankA.status),Communication,Permission)

print('Consent unique identification code: '+str(consentBankA.id))
print('Current consent status: '+str(consentBankA.status))

Consent unique identification code: c0a0ee7c-cffc-4a02-aafb-ba9bd228d8f5
Current consent status: AwaitingAuthorisation


And lets performs the John acceptance action.

In [15]:
john.consents[-1].acceptConsent(john,bankA) # Use -1 to get the most recent consent presented to the Data Subject

Last but not least, lets print the current status of this consent, that should be changed from **AwaitingAuthorisation** to **Authorised**

In [16]:
print('Consent unique identification code: '+str(consentBankA.id))
print('Current consent status: '+str(consentBankA.status))

Consent unique identification code: c0a0ee7c-cffc-4a02-aafb-ba9bd228d8f5
Current consent status: Authorised


In [17]:
print('Data Controller is processing data subject information? '+str(bankA.isProcessing.get(john.id))+'\n')
log.printLog()

Data Controller is processing data subject information? True

Date; Event; Type; Deontic Concept
2022-08-08 16:52:13.393077; A new consent term was generated, but it was not accepted yet ID: c0a0ee7c-cffc-4a02-aafb-ba9bd228d8f5 Current Status: AwaitingAuthorisation; Communication;  Permission
2022-08-08 16:52:13.434455; Consent requested to the Data Subject. ID: c0a0ee7c-cffc-4a02-aafb-ba9bd228d8f5 Current Status: AwaitingAuthorisation; Communication;  Permission
2022-08-08 16:52:13.449068; Data Subject 35512367800 accepted the consent term; Communication;  Permission
2022-08-08 16:52:13.449085; Data Subject can now have all foressen rights; Explanation;  Permission
2022-08-08 16:52:13.449100; Data Controller 00345 can process the Data Subject information from 2022-08-08 to 2027-08-08; Explanation;  Permission


### Scene 2: Data Subject agrees with the consent term.

Now, let's create a consent term where John agrees to share his data from Bank A to Bank B, respecting the Data Regulation specified at the beginning of this document.

In [18]:
consentBankB = Consent()
consentBankB.createConsent('offer_products_and_services','create_specific_offers',
                           'anonymized_data_analytics','SHA256',
                           'Authorized employees can access the data only',
                           'Bank B Private Cloud',
                           bankB,12,12) # LGPD default expiration period is 12 months


john.consents.append(consentBankB)

john.consents[-1].acceptConsent(john,bankB)

In [19]:
log.printLog()

Date; Event; Type; Deontic Concept
2022-08-08 16:52:13.393077; A new consent term was generated, but it was not accepted yet ID: c0a0ee7c-cffc-4a02-aafb-ba9bd228d8f5 Current Status: AwaitingAuthorisation; Communication;  Permission
2022-08-08 16:52:13.434455; Consent requested to the Data Subject. ID: c0a0ee7c-cffc-4a02-aafb-ba9bd228d8f5 Current Status: AwaitingAuthorisation; Communication;  Permission
2022-08-08 16:52:13.449068; Data Subject 35512367800 accepted the consent term; Communication;  Permission
2022-08-08 16:52:13.449085; Data Subject can now have all foressen rights; Explanation;  Permission
2022-08-08 16:52:13.449100; Data Controller 00345 can process the Data Subject information from 2022-08-08 to 2027-08-08; Explanation;  Permission
2022-08-08 16:52:13.481624; A new consent term was generated, but it was not accepted yet ID: cd412226-069a-4c7c-bb38-aff8065c0cb9 Current Status: AwaitingAuthorisation; Communication;  Permission
2022-08-08 16:52:13.481784; Data Subject 35

Additionally, we can check the John's and Bank B rights executing the following code.

### Scene 3: Defining the Data Subject's rights.

As the current sandbox is based on the LGPD, we had to add the rights foreseen to the Data Subject as well as the Data Controller when the Data Subject decides to accept the consent term.

According to the LGPD Art. 18, when the Data Subject is sharing data with a Data Controller, he/she has the following rights:
1. Data Access
2. Data Copy
3. Data Correction
4. Data Anonymization
5. Data Portability
6. Data Deletion
7. Information regarding the data sharing with a third party
8. Request consent revocation.

In [20]:
print('-- Data Subject Rights --')

for right in consentBankB.dataSubjectRights:
  print(right)

print('\n-- Data Controller Rights --')
for right in consentBankB.dataControllerRights:
  print(right)

-- Data Subject Rights --
Data Access
Data Copy
Data Correction
Data Anonymization
Data Portability
Data Deletion
Information regarding the data sharing with a third party
Request consent revocation

-- Data Controller Rights --
Data Processing
Data Collecting
Data Storing


In [21]:
# Auxiliary function to print the transactional data shared in an accepted consent term
for key in consentBankB.transactionalData:
    print(key+': '+ str(consentBankB.transactionalData[key]))


Account Balance: 250
Credit Card Type: Platinum
Limit: 5000
Transactions: {'supermarket': 300, 'library': 700, 'drugstore': 100}


 ---------

### Scene 4: Data Subject's consent revocation.

As mentioned in the scenario's description, the case presented in this sandbox aims to reproduce the consent term revocation consequences, as the Data Subject considered that he does not want to receive offers from Bank B. 

Once performed, the action of requesting the consent revocation cannot be executed again, and the Data Controller is forbidden to still collecting the Data Subject's data.

In [22]:
log.registerAction('Data Subject considered he does not want to receive more offers from Bank B',Communication,Permission)
john.consents[1].revokeConsent(john,bankB)

True

In [23]:
for consent in john.consents:
  print(consent.id,consent.status,consent.dataController.id)

log.printLog()

c0a0ee7c-cffc-4a02-aafb-ba9bd228d8f5 Authorised 00345
cd412226-069a-4c7c-bb38-aff8065c0cb9 Revoked 48634
Date; Event; Type; Deontic Concept
2022-08-08 16:52:13.393077; A new consent term was generated, but it was not accepted yet ID: c0a0ee7c-cffc-4a02-aafb-ba9bd228d8f5 Current Status: AwaitingAuthorisation; Communication;  Permission
2022-08-08 16:52:13.434455; Consent requested to the Data Subject. ID: c0a0ee7c-cffc-4a02-aafb-ba9bd228d8f5 Current Status: AwaitingAuthorisation; Communication;  Permission
2022-08-08 16:52:13.449068; Data Subject 35512367800 accepted the consent term; Communication;  Permission
2022-08-08 16:52:13.449085; Data Subject can now have all foressen rights; Explanation;  Permission
2022-08-08 16:52:13.449100; Data Controller 00345 can process the Data Subject information from 2022-08-08 to 2027-08-08; Explanation;  Permission
2022-08-08 16:52:13.481624; A new consent term was generated, but it was not accepted yet ID: cd412226-069a-4c7c-bb38-aff8065c0cb9 Curr

Therefore, the consent revocation request impacts many LGPD relationships as depicted in Figure 6.

![RootScenarioImpact](./img/RootScenarioImpact.png "Root Scenario Impact")
Fig.6 - Consent Revocation Impact.

The red entities suffered impact directly, or indirectly, when the consent was revoked. First,  the data controller must stop collecting personal data immediately. Next, the data controller must update the sharing policies and access restriction to prevent unauthorized access or new data processing. Still, the consent status will change to "invalid", as the controller cannot use this consent anymore.

 ---------

### Performing explanation exercises regarding possible scenarios

Here, we are going to perform questions regarding access confirmation, rights compliance, and information about consent term. Those questions will exercise the data subject and controller understanding regarding possible scenarios during the relationship between these two actors.


kAre the data controller Bank B using the data subject Paulo's data? 

Expected: As the Data Subject requested to revoke his consent, the data controller is **prohibited** to still using the Data Subjects data.

In [24]:
bankB.isProcessing.get(john.id)

False

Why?

In [25]:
for event in log.root:
  if Prohibition in event:
    print(event)

2022-08-08 16:52:13.526285; Data Subject 35512367800  requested to the Data Controller to revoke his/her consent; Communication;  Prohibition
2022-08-08 16:52:13.526296; From now, the Data Controller 48634 cannot collect the Data Subject information; Communication;  Prohibition
2022-08-08 16:52:13.526304; From now, the Data Controller 48634 cannot process the Data Subject information; Communication;  Prohibition
2022-08-08 16:52:13.526322; From now 2022-08-08, consent cd412226-069a-4c7c-bb38-aff8065c0cb9 is not valid to be used by the data controller; Explanation;  Prohibition


What are the Data Subject rights right now?

Expected: As the Data Subject requested to revoke his consent, he is **prohibited** to create such request again, even though he has **permission** to request the other rights foreseen by the LGPD.

In [26]:
for right in john.consents[1].dataSubjectRights:
  print(right)

Data Access
Data Copy
Data Correction
Data Anonymization
Data Portability
Data Deletion


Who are collecting the Data Subject's personal data and what are the respective data?

In [27]:
for consent in john.consents:
  if DataCollecting in consent.dataControllerRights:
    print(consent.dataController.id, consent.personalData)

00345 {'name': 'John', 'email': 'john@mail.com', 'cellphone': '+1 98638293756'}


Why?

In [28]:
for consent in john.consents:
  print(consent.dataController.id, consent.status)

00345 Authorised
48634 Revoked


As we can see, only Data Controller 00345 (Bank a) is authorized to collect John's data.

Who are storing the Data Subject's sensitive data and what are the respective data?

Expected: Although the Data Subject requested to revoke his consent with Bank B, he did not request for data deletion, so the Data Controller is **permitted** to store his dada.

In [29]:
for consent in john.consents:
  if DataStoring in consent.dataControllerRights:
    print(consent.dataController.id, consent.transactionalData)

00345 {'Account Balance': 250, 'Credit Card Type': 'Platinum', 'Limit': 5000, 'Transactions': {'supermarket': 300, 'library': 700, 'drugstore': 100}}
48634 {'Account Balance': 250, 'Credit Card Type': 'Platinum', 'Limit': 5000, 'Transactions': {'supermarket': 300, 'library': 700, 'drugstore': 100}}


Show all events.

In [30]:
log.printLog()

Date; Event; Type; Deontic Concept
2022-08-08 16:52:13.393077; A new consent term was generated, but it was not accepted yet ID: c0a0ee7c-cffc-4a02-aafb-ba9bd228d8f5 Current Status: AwaitingAuthorisation; Communication;  Permission
2022-08-08 16:52:13.434455; Consent requested to the Data Subject. ID: c0a0ee7c-cffc-4a02-aafb-ba9bd228d8f5 Current Status: AwaitingAuthorisation; Communication;  Permission
2022-08-08 16:52:13.449068; Data Subject 35512367800 accepted the consent term; Communication;  Permission
2022-08-08 16:52:13.449085; Data Subject can now have all foressen rights; Explanation;  Permission
2022-08-08 16:52:13.449100; Data Controller 00345 can process the Data Subject information from 2022-08-08 to 2027-08-08; Explanation;  Permission
2022-08-08 16:52:13.481624; A new consent term was generated, but it was not accepted yet ID: cd412226-069a-4c7c-bb38-aff8065c0cb9 Current Status: AwaitingAuthorisation; Communication;  Permission
2022-08-08 16:52:13.481784; Data Subject 35

----

#### Cause-effect: Reproducing unauthorized use of personal data

Let's picture that the Data Controller did not respect the Data Subject's request and still collecting the Data Subject's data. In such plot, fines must be applied.

<div>
<img src="https://github.com/phalves/ConFIA/blob/main/img/Scenario2.1_Process.png?raw=1" width="600"/>
</div>
Fig.7 - Consent Revocation Scenario Process.

In [37]:
# This command sets that the Data Controller is collecting the Data Subject's data.
bankB.isProcessing[john.id] = True

So, now the data controller is collecting unauthorized data, it was **prohibited** since the data subject requested consent revocation. The following command will check the environment facts and it will insert this fact regarding the rights **violation** in the log.

In [44]:
if bankB.isProcessing[john.id] and (not (john.consents[1].status == Authorised)):
  log.registerAction('Data Controller has no valid consent approved by the Data Subject', Communication, Prohibition)
  log.registerAction('Data Controller is processing unauthorized information', Communication, Violation)

In [43]:
log.printLog()

Date; Event; Type; Deontic Concept
2022-08-08 16:10:04.207304; A new consent term was generated, but it was not accepted yet ID: fa19d60e-85d1-4549-bc0b-5cb8ace703a8 Current Status: AwaitingAuthorisation; Communication;  Permission
2022-08-08 16:10:04.261998; Consent requested to the Data Subject. ID: fa19d60e-85d1-4549-bc0b-5cb8ace703a8 Current Status: AwaitingAuthorisation; Communication;  Permission
2022-08-08 16:10:04.272240; Data Subject 35512367800 accepted the consent term; Communication;  Permission
2022-08-08 16:10:04.272256; Data Subject can now have all foressen rights; Explanation;  Permission
2022-08-08 16:10:04.272272; Data Controller 00345 can process the Data Subject information from 2022-08-08 to 2027-08-08; Explanation;  Permission
2022-08-08 16:10:04.308768; A new consent term was generated, but it was not accepted yet ID: 272a8168-418f-4d44-b2f0-98b1d7e1b365 Current Status: AwaitingAuthorisation; Communication;  Permission
2022-08-08 16:10:04.308972; Data Subject 35

Moreover, Figure 6 may also be used to mitigate if the consent revocation was properly attended. If there is no modification in the red entities, something is not in compliance with LGPD.

----
#### Cause-effect: Data breach, what to do? 

The Data Controller must inform to national authority and to the Data Subject when a data breach occurs that may cause risks or damage to the Data Subject.

Such communication has to be done as soon as possible and should inform:
- personal data category
- what data were leaked
- what were the technical and security measures used to protect data
- the risks related to the incident
- what the data controller will do to revert or mitigate the damage

Depending on the incident severity, the Data Controller will have to disclose such an event in high-impact communication media.

In this sense, lets picture that Data Controller suffered from a hacker attack and Data Subject's personal data were leaked on the social media and he is receiving few calls from different numbers. So, Data Controller is **obligated to** inform the incident to ANPD and inform the Data Subject that his phone number was leaked. 

Even as Data Subject has revoked his consent, he has to be informed regarding the data breach as his data still on the Data Controller's database.

![Scenario2_DataBreach](./img/Scenario2_DataBreach.png "Process 2.2")
Fig.8 - Data Breach Scenario Process.

Thus, let translate this scenario in Prolog facts.

First, once Bank B figure out that there is a data breach, the ANPD and the data subjects involved have to be informed about that.


In [ ]:
log('Data Controller Bank B triggered an alert to ANPD and to all data subjects affected by
    the data breach informing that all e-mails were exposed','Communication','Obligation',1638970870).

Next, Bank B has to explain that they had adopted security actions to avoid data breach. 

In [ ]:
log('Data Controller Bank B informed the security measures to do not let data breach occurs',
    'Communication','Obligation',1638970871).

Then, Bank B fixed the vulnerability and inform the data subjects as well.

In [ ]:
log('Data Controller Bank B informed that the vulnerability was found 
    and there is no unauthorized access anymore','Communication','Compliance',1638970872).

Furthermore, Bank B inform to Data Subjects that there is a technical group available to help anyone that have had troubles caused by this incident.

In [ ]:
log('Data Controller Bank B created a technical team to help any data subject 
    that have had issues with this incident','Communication','Obligation',1638970873).

As the log shows, this case can present many different ends depending on the damage caused to the data subjects involved. Here, as the data subject received just a few calls and there was low damage, he decided not to enter in dispute to get reparation compensations, even though the data controller has **violated** the Data Subject's privacy.

Moreover, the omission of any fact related to informing the Data Subjects about unauthorized access or neglect the system security, fines should be applied to the Data Controller.

Last but not least, if the Data Controller noticed a data breach, once informed, the Data Controller has to act immediately. **LGPD Art. 48**

In [ ]:
?- log(Event,Type,DeonticOperator,Date) {-1}.

Event = Data Subject can ask if the Data Controller is processing his/her data, Type = Explanation, DeonticOperator = Permission, Date = 1638970860 ;
Event = Data Subject verified the consent term and it was ok, Type = Explanation, DeonticOperator = Obligation, Date = 1638970860 ;
Event = Data Subject agrees with consent term, Type = Communication, DeonticOperator = Compliance, Date = 1638970860 ;
Event = Data Controller can collect the Data Subject information, Type = Explanation, DeonticOperator = Permission, Date = 1638970860 ;
Event = Data Controller can store the Data Subject information, Type = Explanation, DeonticOperator = Permission, Date = 1638970860 ;
Event = Data Controller can process the Data Subject information, Type = Explanation, DeonticOperator = Permission, Date = 1638970860 ;
Event = Data Subject can now have all foressen rights, Type = Explanation, DeonticOperator = Permission, Date = 1638970860 ;
Event = Data Subject considered he does not want to receive more off

Therefore, the data breach impacts many LGPD relationships as depicted in Figure 9.


![DataBreachImpact](./img/DataBreachImpact.png "Data Breach Impact")
Fig.9 - Data Breach Impact.

First, as mentioned before, a data breach event must be informed to all agents impacted. This message must contain the security methods and the storage technologies applied to avoid a data breach. However, this is an event that could trigger other impacts. For instance, after a data breach, the data subject could enter in a dispute resolution claiming discrimination, loss, and unauthorized uses of his/her data. 

Furthermore, the Data Subject might request changes in the consent term, impacting the sharing policies and access restriction. Also, the Data Subject might request consent revocation, data deletion, which affects data collecting, processing, and storing.

In [ ]:
% Resetting scenario
?- retract(log('Data Controller Bank B triggered an alert to ANPD and to all data subjects affected by
    the data breach informing that all e-mails were exposed','Communication','Obligation',1638970870)).
    
?- retract(log('Data Controller Bank B informed the security measures to do not let data breach 
    occurs','Communication','Obligation',1638970871)).
                        
?- retract(log('Data Controller Bank B informed that the vulnerability was found 
    and there is no unauthorized access anymore','Communication','Compliance',1638970872)).
    
?- retract(log('Data Controller Bank B created a technical team to help any data subject 
    that have had issues with this incident','Communication','Obligation',1638970873)).

true.
true.
true.
true.

----
#### Cause-effect: How to get evidences that the Data Controller leaked the Data Subject's data? 

To create concrete evidence that a Data Controller leaked a Data Subject's data, first, it is important to verify who has such data. If there is just one Data Controller legally storing such data; hence, the chances of such Data Controller had leaked personal data is elevated.

Moreover, the data controller is **obligated** to inform if personal or sensitive data is stored in the database. The data subject can request such information for each data controller.

However, as Bank A has the same data as Bank B, the action of verifying who leaked the data is even more difficult and may turns the process below inconclusive.

![Scenario2.2_Process](./img/Scenario2.2_Process.png "Process 2.2")
Fig.10 - Data Leak Process.

In summary, it is important to the data subject to check the consent term in order to verify if there is any clause/ condition which permits the data controller to share data with others. If the data subject disagrees with such clause, he is **permitted** to revoke the consent term anytime.

In [ ]:
% This command sets that Bank A, i.e., the origin instituion, has the same data as Bank B.

dcIsStoringDSData(id(20),dataController('Bank A'),dataSubject('John'),
                    personalData('John','john@mail.com'),sensitiveData('John','transactional_data'),
                    startDate(1638970860),endDate(1670506860)).

In [ ]:
% This command verifies who is storing John's personal and health data.

?- dcIsStoringDSData(id(ID),dataController(DataController),dataSubject('John'),
                    personalData('John','john@mail.com'),sensitiveData('John','transactional_data'),
                    startDate(1638970860),endDate(1670506860)).

ID = 10, DataController = Bank B ;
ID = 20, DataController = Bank A .

Now, let's picture that the Data Controller Institution ABC has John's data.

In [ ]:
dcIsStoringDSData(id(null),dataController('Institution ABC'),dataSubject('John'),
                    personalData('John','john@mail.com'),sensitiveData('John','transactional_data'),
                    startDate(1638970860),endDate(1670506860)).

In [ ]:
% These commands:
%   (i) verifies if the Data Controller is storing the Data Subject's data;
%  (ii) verifies if there is any evidence that the Data Subject allowed the Data Controller to process his/her data;
% (iii) if all previous verifications are true, insert in the log that the Data Controller is not allowed 
%       to collect the Data Subject's data.

%(i)
?- dcIsStoringDSData(id(null),dataController('Institution ABC'),dataSubject('John'),
                    personalData('John','john@mail.com'),sensitiveData('John','transactional_data'),
                    startDate(1638970860),endDate(1670506860)).

%(ii)
?- dsAgreeWithConsentTerms(dataSubject('Institution ABC'),dataController('John'),startDate(1638970860),endDate(1670506860)).
   
%(ii)
?- consentTermStatus(id(10),dataController('Bank B'),dataSubject('John'),status('Valid')).

%(iii)
?- assertz(log('Data Subject did not agree with Institution ABC consent term, so the data was improperly collected, 
    fines should be applied','Explanation','Violation',1638970861)).

true.
false.
false.
true.

Therefore, the event log will show that there is not consent agreement between John and InstitutionABC. 

Hence, Institution ABC is **prohibited** to use such data, i.e., the data was improperly collected. 
Moreover, as Bank A and Bank B are the only Data Controller storing John's data, probably Bank A or Bank B **violated**, on purpose or not, the consent term and the data were leaked from Bank A or Bank B to Institution ABC.

Moreover, this scenario presents the same possible impacts depicted in Figure 9 of the previous plot.

In [ ]:
?- log(Event,Type,DeonticOperator,Date) {-1}.

Event = Data Subject can ask if the Data Controller is processing his/her data, Type = Explanation, DeonticOperator = Permission, Date = 1638970860 ;
Event = Data Subject verified the consent term and it was ok, Type = Explanation, DeonticOperator = Obligation, Date = 1638970860 ;
Event = Data Subject agrees with consent term, Type = Communication, DeonticOperator = Compliance, Date = 1638970860 ;
Event = Data Controller can collect the Data Subject information, Type = Explanation, DeonticOperator = Permission, Date = 1638970860 ;
Event = Data Controller can store the Data Subject information, Type = Explanation, DeonticOperator = Permission, Date = 1638970860 ;
Event = Data Controller can process the Data Subject information, Type = Explanation, DeonticOperator = Permission, Date = 1638970860 ;
Event = Data Subject can now have all foressen rights, Type = Explanation, DeonticOperator = Permission, Date = 1638970860 ;
Event = Data Subject considered he does not want to receive more off

In [ ]:
% Resetting scenario
?- retract(log('Data Subject did not agree with Institution ABC consent term, so the data was improperly collected, 
    fines should be applied','Explanation','Violation',1638970861)),
    
    retract(dcIsStoringDSData(id(null),dataController('Institution ABC'),dataSubject('John'),
                    personalData('John','john@mail.com'),sensitiveData('John','transactional_data'),
                    startDate(1638970860),endDate(1670506860))).

true.

----
#### Cause-effect: Requesting data correction

Data correction is one of the Data Subject's right foreseen in LGPD in the moment that the consent term was accepted.
Even if the Data Subject revoke his/her consent, the data will not be deleted; a express data deletion request is required.

So, in order to check if the data corretion request was accomplished, the Data Subject should call another right - data access.

The data controller is **obligated** to abide by the data subjects' requests as correction as data access.
Also, the controller is **obligated** to inform all processors regarding the correction.


<div>
<img src="https://github.com/phalves/ConFIA/blob/main/img/Scenario2.3_Process.png?raw=1" width="600"/>
</div>
Fig.11 - Data Correction Process.

First, the Data Subject should verify if the Data Controller is storing his/her data.

In [ ]:
?- dcIsStoringDSData(id(ID),dataController('Bank B'),dataSubject('John'),
                    personalData('John','john@mail.com'),sensitiveData('John','transactional_data'),
                    startDate(1638970860),endDate(1670506860)).

ID = 10 .

If true, the Data Subject should have the right to data access and data correction.

In [ ]:
?- dsRight(dataAccess,dataSubject('John'),dataController('Bank B')),
    dsRight(dataCorrection,dataSubject('John'),dataController('Bank B')).

true.

Then, the Data Subject is able to request and verify if the data was changed.

In [ ]:
log('Data Subject requested to change his e-mail criptojohn@mail.com','Communication','Permission',1638970866).

And the Data Controller executed this correction.

In [ ]:
% First, the Data Controller verifies if the Data Subject has the rights requiered to perform such action.
% Then, remove the incorrect data and insert the new data.


?-  dsRight(dataCorrection,dataSubject('John'),dataController('Bank B')),

    retract(dcIsStoringDSData(id(10),dataController('Bank B'),dataSubject('John'),
                    personalData('John','john@mail.com'),sensitiveData('John','transactional_data'),
                    startDate(1638970860),endDate(1670506860))),

    assertz(dcIsStoringDSData(id(10),dataController('Bank B'),dataSubject('John'),
                    personalData('John','criptojohn@mail.com'),sensitiveData('John','transactional_data'),
                    startDate(1638970860),endDate(1670506860))),
    
    assertz(log('Data Controller has to execute the Data Subject s requerst','Explanation','Obligation',1638970866)),
        
    assertz(log('Data Controller changed the data as requested by the Data Subject','Communication','Compliance',1638970867)),
    
    assertz(log('Data Controller notified all processors regarding the data corretion','Communication','Compliance',1638970867)).

true.

Hence, as the data controler attended the data subject's request, it still in **compliance** with the LGPD. The Data Subject can verify if the data was fixed.

In [ ]:
% If the Data Subject has the right to access his/her data, then he/she is able to verify if his/her data was fixed.

?-  dsRight(dataAccess,dataSubject('John'),dataController('Bank B')),
    dcIsStoringDSData(id(10),dataController(DataController),dataSubject('John'),
                    personalData('John',Email),sensitiveData('John','transactional_data'),
                    startDate(1638970860),endDate(1670506860)),
    assertz(log('Data Subject confirmed that the data was fixed','Explanation','Permission',1638970868)).

DataController = Bank B, Email = criptojohn@mail.com .

In [ ]:
?- log(Event,Type,DeonticOperator,Date), Date >= 1638970866 {-1}.

Event = Data Subject requested to change his e-mail criptojohn@mail.com, Type = Communication, DeonticOperator = Permission, Date = 1638970866 ;
Event = Data Controller has to execute the Data Subject s requerst, Type = Explanation, DeonticOperator = Obligation, Date = 1638970866 ;
Event = Data Controller changed the data as requested by the Data Subject, Type = Communication, DeonticOperator = Compliance, Date = 1638970867 ;
Event = Data Controller notified all processors regarding the data corretion, Type = Communication, DeonticOperator = Compliance, Date = 1638970867 ;
Event = Data Subject confirmed that the data was fixed, Type = Explanation, DeonticOperator = Permission, Date = 1638970868 .

Therefore, the data correction impacts many LGPD relationships as depicted in Figure 12.

![DataCorrectionImpact](./img/DataCorrectionImpact.png "Data Correction Impact")
Fig.12 - Data Correction Impact.

The data correction impacts data processing and data storage, as the personal or sensitive data were changed. Therefore, Data Controllers and Processors should also verify if the copy requested by the Data Subject is the updated personal and sensitive data.

In [ ]:
% Resetting scenario
?-  retract(log('Data Subject requested to change his e-mail criptojohn@mail.com','Communication','Permission',1638970866)),
    retract(log('Data Controller has to execute the Data Subject s requerst','Explanation','Obligation',1638970866)),
    retract(log('Data Controller changed the data as requested by the Data Subject','Communication','Compliance',1638970867)),
    retract(log('Data Subject confirmed that the data was fixed','Explanation','Permission',1638970868)),
    retract(log('Data Controller notified all processors regarding the data corretion','Communication','Compliance',1638970867)),
    retract(dcIsStoringDSData(id(10),dataController('Bank B'),dataSubject('John'),
                    personalData('John','criptojohn@mail.com'),sensitiveData('John','transactional_data'),
                    startDate(1638970860),endDate(1670506860))).
    %assertz(dcIsStoringDSData(id(10),dataController(universityXYZ),dataSubject('John'),
    %                personalData('John','john@mail.com'),sensitiveData('John','transcripts'),
    %                startDate(1622035260),endDate(1637846460))).

true.

----
#### Cause-effect: Requesting anonymization

Let's picture that the Data Subject requested the data anonymization right. Once the data is anonymized, the Data Controller will not have the resources to give any details about such data, including correction. Hence, after this request, the data controller is **not obligated** to comply with requests that should involve reidentification actions.

Also, the controller is **obligated** to inform all processors regarding the anonymization.

Here, questions regarding the anonymization algorithms could emerge, but this is not the focus of this work.
This work focus on the causes and consequences understanding of possible scenarios.

TAG \[Domain specificity]: 
In the open banking scenario, the data anonymization request may implies in stop receiving offers and products from Bank B.

In [ ]:
?- checkConsentTerm(id(11),dataController('Bank B'),
                        dataSubject('John'),
                        purpose('Bank B','offer_products_and_services'),
                        'create_specific_offers',
                        'data_analytics',
                        31536000,
                        'none',
                        'e-mail',
                        'lgpd@bankb.br',
                        1639970860).                        

true.

In [ ]:
?- setThatdsAgreeWithConsentTerms(id(11),
                                dataSubject('John'),
                                dataController('Bank B'),
                                requestFormat('Direct','John','null'),
                                personalData('John','john@mail.com'),
                                sensitiveData('John','transactional_data'),
                                startDate(1639970860),
                                endDate(EndDate)), EndDate is 1639970860+31536000.

EndDate = 1671506860 .

In [ ]:
?- setDSRights(dataSubject('John'),dataController('Bank B'),startDate(1639970860)).

true.

Once the data is anonymized, the Data Controller will not have the resources to give any details about such data, including correction. Hence, if the Data Subject submits this request, he will not be able to participate in this discipline. The data controller is **not obligated** to comply with requests that should involve deidentification actions.

Also, the controller is **obligated** to inform all processors regarding the anonymization.

Here, questions regarding the anonymization algorithms could emerge, but this is not the focus of this work.
This work focus on the causes and consequences understanding of possible scenarios.


<div>
<img src="https://github.com/phalves/ConFIA/blob/main/img/Scenario2.4_Process.png?raw=1" width="600"/>
</div>
Fig.13 - Data Anonymization Process.

First, the Data Subject should show that the Data Controller has his data.

TAG \[Domain specificity]: 
- As the anonymization right turns not possible for Bank B to execute its purpose, i.e., create and send specific products and services based on the Data Subject data, Bank B should ask if the Data Subject would like to revoke his consent. However, one of the very first steps in this scenario description is the consent revocation action; hence, the data subject already revoked his consent. 

In [ ]:
?- dcIsStoringDSData(id(11),dataController('Bank B'),dataSubject('John'),
                        personalData('John','john@mail.com'),sensitiveData('John','transactional_data'),
                        startDate(1639970860),endDate(1671506860)),
                    
    assertz(log('Data Subject requested to anonymize his data','Communication','Permission',1639970862)).

true.

Next, the Data Controller accomplish the Data Subject request.

In [ ]:
?- retract(dcIsStoringDSData(id(11),dataController('Bank B'),dataSubject('John'),
                        personalData('John','john@mail.com'),sensitiveData('John','transactional_data'),
                        startDate(1639970860),endDate(1671506860))),

    assertz(dcIsStoringDSData(id(_),dataController('Bank B'),dataSubject(_),
                    personalData(_,_),sensitiveData(_,'anonymized_transactional_data'),
                    startDate(1639970860),endDate(1671506860))),

    assertz(log('Data Controller has to execute the Data Subject s request','Delete-Anonymise','Compliance',1671506863)),
    assertz(log('Data Controller has to notify all processors regarding the anonymization request','Delete-Anonymise','Compliance',1671506863)),
    assertz(log('Data Controller anonymized the Data Subjects data','Delete-Anonymise','Compliance',1671506863)),
    assertz(log('Data Controller notified the Data Subject that the purpose will not be executed anymore','Delete-Anonymise','Compliance',1671506863)),    
    assertz(log('Data Subject cannot request data: access, copy, correction,anonymization, 
            portability, deletion, and details of data sharing','Communication','Prohibition',1671506863)),
            
    assertz(log('Data Subject does not accept to share his information anymore','Communication','Prohibition',1671506863)),
    
    retract(dsRight(dataAccess,dataSubject('John'),dataController('Bank B'))),
    retract(dsRight(dataCopy,dataSubject('John'),dataController('Bank B'))),
    retract(dsRight(dataCorrection,dataSubject('John'),dataController('Bank B'))),
    retract(dsRight(dataAnonymization,dataSubject('John'),dataController('Bank B'))),
    retract(dsRight(dataPortability,dataSubject('John'),dataController('Bank B'))),
    retract(dsRight(dataDeletion,dataSubject('John'),dataController('Bank B'))),
    retract(dsRight(dataSharingInformation,dataSubject('John'),dataController('Bank B'))).

 ;
 .

Now, the Data Subject's sensitive data is not associated with any personal data that could identify that such data is from the Data Subject. Hence, the data controller is in **compliance** with LGPD.

In [ ]:
?- dcIsStoringDSData(id(_),dataController('Bank B'),dataSubject(DataSubject),
                    personalData(_,_),sensitiveData(_,'anonymized_transactional_data'),
                    startDate(1639970860),endDate(1671506860)).

DataSubject = Variable(70) .

In [ ]:
?- log(Event,Type,DeonticOperator,Date), Date >= 1639970862 {-1}.

Event = Data Subject requested to anonymize his data, Type = Communication, DeonticOperator = Permission, Date = 1639970862 ;
Event = Data Controller has to execute the Data Subject s request, Type = Delete-Anonymise, DeonticOperator = Compliance, Date = 1671506863 ;
Event = Data Controller has to notify all processors regarding the anonymization request, Type = Delete-Anonymise, DeonticOperator = Compliance, Date = 1671506863 ;
Event = Data Controller anonymized the Data Subjects data, Type = Delete-Anonymise, DeonticOperator = Compliance, Date = 1671506863 ;
Event = Data Controller notified the Data Subject that the purpose will not be executed anymore, Type = Delete-Anonymise, DeonticOperator = Compliance, Date = 1671506863 ;
Event = Data Subject cannot request data: access, copy, correction,anonymization, portability, deletion, and details of data sharing, Type = Communication, DeonticOperator = Prohibition, Date = 1671506863 ;
Event = Data Subject does not accept to share his info

Furthermore, the remaining rights are: the right to ask if a Data Controller is processing his/her data, and the right to request consent revocation .

In [ ]:
?- dsRight(RIGHT,dataSubject('John'),dataController('Bank B')).

RIGHT = processingConfirmation ;
RIGHT = requestConsentRevocation .

Therefore, the data anonymization impacts many LGPD relationships, as depicted in Figure 14.

![DataAnonymizationImpact](./img/DataAnonymizationImpact.png "Data Anonymization Impact")
Fig.14-  Data anonymization Impact.

Data anonymization impacts almost all Data Subjects Rights. The anonymization process may turn the personal data not identifiable anymore. Hence, the anonymized data is out of LGPD's scope. In this sense, requests related to data access, deletion, correction, portability, or copy, may not be answered by the Data Controller, as the Controller might not identify the Data Subject anymore.

Last but not least, there are other scenarios that could be explored as well.

TAG \[Domain specificity]: 
 -  For instance, if the data subject requested anonymization and not did not request consent revocation, the new data will not be anonymized.

In [ ]:
% Resetting scenario
?-  assertz(dcIsStoringDSData(id(11),dataController('Bank B'),dataSubject('John'),
                        personalData('John','john@mail.com'),sensitiveData('John','transactional_data'),
                        startDate(1639970860),endDate(1671506860))),
    retract(dcIsStoringDSData(id(_),dataController('Bank B'),dataSubject(_),
                    personalData(_,_),sensitiveData(_,'anonymized_transactional_data'),
                    startDate(1639970860),endDate(1671506860))),
    retract(log('Data Subject requested to anonymize his data','Communication','Permission',1639970862)),
    retract(log('Data Controller has to execute the Data Subject s request','Delete-Anonymise','Compliance',1671506863)),
    retract(log('Data Controller has to notify all processors regarding the anonymization request','Delete-Anonymise','Compliance',1671506863)),
    retract(log('Data Controller anonymized the Data Subjects data','Delete-Anonymise','Compliance',1671506863)),
    retract(log('Data Controller notified the Data Subject that the purpose will not be executed anymore','Delete-Anonymise','Compliance',1671506863)),
    retract(log('Data Subject cannot request data: access, copy, correction,anonymization, 
            portability, deletion, and details of data sharing','Communication','Prohibition',1671506863)),
    retract(log('Data Subject does not accept to share his information anymore','Communication','Prohibition',1671506863)),
    assertz(dsRight(dataAccess,dataSubject('John'),dataController('Bank B'))),
    assertz(dsRight(dataCopy,dataSubject('John'),dataController('Bank B'))),
    assertz(dsRight(dataCorrection,dataSubject('John'),dataController('Bank B'))),
    assertz(dsRight(dataAnonymization,dataSubject('John'),dataController('Bank B'))),
    assertz(dsRight(dataPortability,dataSubject('John'),dataController('Bank B'))),
    assertz(dsRight(dataDeletion,dataSubject('John'),dataController('Bank B'))),
    assertz(dsRight(dataSharingInformation,dataSubject('John'),dataController('Bank B'))),
    retract(consentTermStatus(id(11),dataController('Bank B'),dataSubject('John'),status('Valid'))),
    assertz(consentTermStatus(id(11),dataController('Bank B'),dataSubject('John'),status('Invalid'))).

true.

----
#### Cause-effect: Data deletion

As mentioned before, there is more than one definition for data deletion. In the case aforementioned, we anonymized the data, which can be considered as data deletion. Now, let's picture that the Data Subject John wants to destroy his data from the Bank's B database. 

This means that John will not be able to subscribe in any class from Bank B.

Moreover, the following purposes, also described in LGPD art. 16, legitimizes, i.e., **allow** the Data Controller to keep the personal data stored in the database:
 - I - compliance with a legal or regulatory obligation by the controller;
 - II - study by a research institution, ensuring, whenever possible, the anonymization of personal data;
 - III - transfer to a third party, provided that the data processing requirements set out in this Law is respected; or
 - IV - exclusive use of the controller, its access by a third party is prohibited, and anonymization is required as well.


<div>
<img src="https://github.com/phalves/ConFIA/blob/main/img/Scenario2_DataDeletion.png?raw=1" width="600"/>
</div>
Fig.15 - Data Deletion Process.

As the Bank B purpose is not based on any aforementioned situations; hence, Bank B must accept the Data Subject request.

In [ ]:
?- purpose('Bank B','John',Purpose).

Purpose = offer_products_and_services .

In order to specify the rule that defines if the Data Controller has the right to keep the Data Subject's data, we developed the following function.

In [ ]:
% Description: This function verifies if the Data Controller purpose is elegible to hold the Data Subject's data.
% This function receives the params:
%  i. Data Subject
% ii. Data Controller

verifyIfDCCanHoldDSData(dataController(DC),dataSubject(DS)) :-
    purpose(DC,DS,legalObligation), assertz(dcCanHoldData(dataController(DC),dataSubject(DS)));
    purpose(DC,DS,research), assertz(dcCanHoldData(dataController(DC),dataSubject(DS)));
    purpose(DC,DS,transferToThirdParty), assertz(dcCanHoldData(dataController(DC),dataSubject(DS)));
    purpose(DC,DS,exclusiveDCUse), assertz(dcCanHoldData(dataController(DC),dataSubject(DS)));
    assertz(dcCanHoldData('','')).

So, let's run the above function to verify if a new fact is generated informing that the Data Controller can hold de Data Subject's data.

In [ ]:
?- verifyIfDCCanHoldDSData(dataController('Bank B'),dataSubject('John')). 

true.

Then, let's verify if such a fact was generated.

In [ ]:
?- dcCanHoldData(dataController('Bank B'),dataSubject('John')).

false.

The result is 'false' because the purpose is different from the purposes listed by LGPD that allows the Data Controller keeps the data.
In this sense, let's simulate the request for data deletion from the Data Subject John to the Data Controller Bank B.

In [ ]:
log('Data Subject requested to delete his data','Communication','Permission',1639970864).

In [ ]:
log('Data Controller received the data deletion request and will evaluate the solicitation','Communication','Compliance',1639970865).

The code below will check if the Data Controler can hold the Data Subject information. If yes, two new log activities will be recorded. On the other hand, the code will return false if the Data Controller cannot hold the information, and the two log activities will not be recorded.

In [ ]:
?- dcCanHoldData(dataController('Bank B'),dataSubject('John')),
    
    assertz(log('Data Controller can hold the data because its purpose allows it.','Explanation','Permission',1639970866)),

    assertz(log('Data Controller decided to keep the data on the database.','Communication','Compliance',1639970867)).

false.

In [ ]:
% Description: This function deletes the Data Subject's data if the Data Controller is allowed to do that.
% This function receives the params:
%    i. Data Subject
%   ii. Data Controller
%  iii. Date Time

requestToDeleteDSData(dataSubject(DS),dataController(DC),date(DT)) :- 
 not(dcCanHoldData(dataController(DC),dataSubject(DS))),
 assertz(log('Data Controller has to delete the data','Communication','Obligation',DT)).

In [ ]:
?- requestToDeleteDSData(dataSubject('John'),dataController('Bank B'),date(1639970868)).

true.

Why 1: This method returned true because the Data Controller **cannot keep** the Data Subject's data, as presented bellow.

In [ ]:
?-  dcCanHoldData(dataController('Bank B'),dataSubject('John')).

false.

Why 2: The Data Controller cannot keep personal data because it will be not used for any aforementioned situations.

In [ ]:
?- purpose(DC,'John',Purpose).

DC = Bank B, Purpose = offer_products_and_services .

It means that the purpose presented by the Data Controller is **not valid** to permit that Bank B holds the Data Subject's data.

In [ ]:
?- log(Event,Type,DeonticOperator,Date), Date >= 1639970864 {-1}.

Event = Data Subject requested to delete his data, Type = Communication, DeonticOperator = Permission, Date = 1639970864 ;
Event = Data Controller received the data deletion request and will evaluate the solicitation, Type = Communication, DeonticOperator = Compliance, Date = 1639970865 ;
Event = Data Controller has to delete the data, Type = Communication, DeonticOperator = Obligation, Date = 1639970868 .

Therefore, as depicted in Figure 16, the request for data deletion may impacts differently depending on the purpose limitation. For example, the data storage may have to anonymize the data. Moreover, if the data was deleted or anonymized, the Data Controller cannot achieve requests related to data correction, portability, and copy anymore.

![DataDeletionImpact](./img/DataDeletionImpact.png "Data Deletion Impact")
Fig.16 - Data Deletion Impact.

In [ ]:
% Resetting scenario
?-  retract(log('Data Subject requested to delete his data','Communication','Permission',1639970864)),
    retract(log('Data Controller received the data deletion request and will evaluate the solicitation',
        'Communication','Compliance',1639970865)),
    retract(log('Data Controller has to delete the data','Communication','Obligation',1639970868)).

true.

----
#### Cause-effect: Technology unavailability

Companies are vulnerable to technical fault, unavailability, or security breach. In this sense, Data Subjects might be impacted by technology throubles. In some cases, the technology unavailability may not impact Data Subjects, but only internal companies' processes.

In this scenario, we will simulate an event of technology unavailability, i.e., let's suppose that Bank B's cloud server, which has personal data storage, is offline. Internally, Bank B suffered a high impact of this unavailability; all systems that depend on this database are offline, i.e., the internal data governance is jeopardized/ compromised. Hence, employees and clients cannot access any internal system.

Figure 17 depicts the impact of system unavailability.

![DataUnavailabilityImpact](./img/DataUnavailabilityImpact.png "Data Unavilability Impact")
Fig.17 - Data Unavailability Impact.

Besides the governance, data unavailability may impacts directly the users' rights. For example, the Data Controller and Processor cannot delete or execute data corrections if the system is unavailable. Moreover, if a Data Controller requests for portability, anonymization, or portability, the Data Controller will not be able to attend to such requests as fast as expected; a considerable delay is expected, instead. Furthermore, fines can be applied depending on the delay, but they should be evaluated case-by-case.

Last but not least, in this scenario, depending on the moment of this data unavailability, the impact might be more, or less, severe. For instance, if a client wants to subscribe to a service that will improve the client's profit, he/she will probably be affected if the operation does not finalize. As a result, the client could lose the yield of such a day.

Conversely, if the unavailability occurs on the weekend, the clients may not be impacted.

----
#### Cause-effect: Inconsistent behavior

Forecast human behavior can be a big challenge; the strange behaviors can be originated from different aspects, such as by a bad interface, system instability or error, or by a malicious person. In this sense, let's picture a person who agrees and revokes consent repeatedly in short time-space.

TAG \[Domain specificity]: In the open banking scenario, this behavior might indicate that the client is confused regarding the process of sharing his/her information, or the client is trying to manipulate the data to get benefits.

In [ ]:
?- createConsentTerm(12,'Bank B','John','john@mail.com','personal_and_transactional','offer_products_and_services',
                'create_specific_offers',
                'data_analytics',
                31536000,
                'none',
                'e-mail',
                'lgpd@bankb.br',
                'SHA256',
                'Authorized employees can access the data only',
                'Bank B private cloud'),
                
    assertz(dsRight(processingConfirmation,dataSubject('John'),dataController('Bank B'))),
    assertz(log('Data Subject can ask if the Data Controller is processing his/her data','Explanation','Permission',1639970870)),

    checkConsentTerm(id(12),dataController('Bank B'),
                        dataSubject('John'),
                        purpose('Bank B','offer_products_and_services'),
                        'create_specific_offers',
                        'data_analytics',
                        31536000,
                        'none',
                        'e-mail',
                        'lgpd@bankb.br',
                        1638970860),    
                        
    setThatdsAgreeWithConsentTerms(id(12),
                                dataSubject('John'),
                                dataController('Bank B'),
                                requestFormat('Direct','John','null'),
                                personalData('John','john@mail.com'),
                                sensitiveData('John','transactional_data'),
                                startDate(1639970870),
                                endDate(EndDate)), EndDate is 1639970870+31536000.
                        
    setDSRights(dataSubject('John'),dataController('Bank B'),startDate(1639970870)).

EndDate = 1671506870 ;
EndDate = 1671506870 ;
EndDate = 1671506870 ;
EndDate = 1671506870 ;
EndDate = 1671506870 ;
EndDate = 1671506870 ;
EndDate = 1671506870 ;
EndDate = 1671506870 ;
EndDate = 1671506870 ;
EndDate = 1671506870 .

In [ ]:
?- setDSRevokeConsent(id(12),
                        dataSubject('John'),
                        dataController('Bank B'),
                        personalData('John','john@mail.com'),
                        sensitiveData('John','transactional_data'),
                        now(1639970872),
                        startDate(1639970870),
                        endDate(EndDate)
                        ), 
                        EndDate is 1639970870+31536000.

EndDate = 1671506870 ;
EndDate = 1671506870 ;
EndDate = 1671506870 ;
EndDate = 1671506870 .

In [ ]:
?- createConsentTerm(13,'Bank B','John','john@mail.com','personal_and_transactional','offer_products_and_services',
                'create_specific_offers',
                'data_analytics',
                31536000,
                'none',
                'e-mail',
                'lgpd@bankb.br',
                'SHA256',
                'Authorized employees can access the data only',
                'Bank B private cloud').

true.

In [ ]:
?- assertz(dsRight(processingConfirmation,dataSubject('John'),dataController('Bank B'))),
    assertz(log('Data Subject can ask if the Data Controller is processing his/her data','Explanation','Permission',1639970872)).

true.

In [ ]:
?- checkConsentTerm(id(13),dataController('Bank B'),
                        dataSubject('John'),
                        purpose('Bank B','offer_products_and_services'),
                        'create_specific_offers',
                        'data_analytics',
                        31536000,
                        'none',
                        'e-mail',
                        'lgpd@bankb.br',
                        1639970872).    

 ;
 ;
 ;
 ;
 ;
 ;
 ;
 ;
 ;
 .

In [ ]:
?- setThatdsAgreeWithConsentTerms(id(13),
                                dataSubject('John'),
                                dataController('Bank B'),
                                requestFormat('Direct','John','null'),
                                personalData('John','john@mail.com'),
                                sensitiveData('John','transactional_data'),
                                startDate(1639970872),
                                endDate(EndDate)), EndDate is 1639970870+31536000.

EndDate = 1671506870 ;
EndDate = 1671506870 ;
EndDate = 1671506870 ;
EndDate = 1671506870 ;
EndDate = 1671506870 ;
EndDate = 1671506870 ;
EndDate = 1671506870 ;
EndDate = 1671506870 ;
EndDate = 1671506870 ;
EndDate = 1671506870 .

In [ ]:
?- setDSRights(dataSubject('John'),dataController('Bank B'),startDate(1639970872)).

true.

In [ ]:
?- setDSRevokeConsent(id(13),
                        dataSubject('John'),
                        dataController('Bank B'),
                        personalData('John','john@mail.com'),
                        sensitiveData('John','transactional_data'),
                        now(1639970872),
                        startDate(1639970872),
                        endDate(EndDate)
                        ), 
                        EndDate is 1639970872+31536000.

EndDate = 1671506872 ;
EndDate = 1671506872 ;
EndDate = 1671506872 ;
EndDate = 1671506872 ;
EndDate = 1671506872 ;
EndDate = 1671506872 ;
EndDate = 1671506872 ;
EndDate = 1671506872 ;
EndDate = 1671506872 ;
EndDate = 1671506872 .

Figure 18 depicts the impacted entities.

![InconsistentBehaviorImpact](./img/InconsistentBehaviorImpact.png "Inconsistent Behaviour Impact")
Fig.18 - Inconsistent Behavior Impact.

This unusual behavior can be caught by analyzing the event log. Depending on the magnitude, this kind of event may cause damage to the system, as the Deny of Service attack. 

Moreover, in our scenario, as Bank B needs the client's data to offer products and services, the client might perform such inconsistent behavior to try getting advantages hiding specific data which may compromise his/her reputation. In this case, if Bank B identifies such behavior, the bank may request additional information before starting offering new products and services.

In [ ]:
?- log(Event,Type,DeonticOperator,Date), Date >= 1639970870 {-1}.

Event = Data Subject can ask if the Data Controller is processing his/her data, Type = Explanation, DeonticOperator = Permission, Date = 1639970870 ;
Event = Data Subject agrees with consent term, Type = Communication, DeonticOperator = Compliance, Date = 1639970870 ;
Event = Data Controller can collect the Data Subject information, Type = Explanation, DeonticOperator = Permission, Date = 1639970870 ;
Event = Data Controller can store the Data Subject information, Type = Explanation, DeonticOperator = Permission, Date = 1639970870 ;
Event = Data Controller can process the Data Subject information, Type = Explanation, DeonticOperator = Permission, Date = 1639970870 ;
Event = Data Subject agrees with consent term, Type = Communication, DeonticOperator = Compliance, Date = 1639970870 ;
Event = Data Controller can collect the Data Subject information, Type = Explanation, DeonticOperator = Permission, Date = 1639970870 ;
Event = Data Controller can store the Data Subject information, Type =

----
#### Cause-effect: Data portability


Data portability can be explored at least in two ways. First, as cellphone companies, data portability means migrating the Data Subject phone number to another company. The client information should be migrated from one company to another. Second, like streaming video companies, data portability may mean just the act of copying the data to another company. Both companies would have the same client data at the moment of data portability request. 

As the open banking scenario already acts as a kind of data portability request and exercises data sharing and consent revocation, such a cause-effect scenario will not be explored here.

----
#### Cause-effect: Plug a new consent term


Pluggable consent is a non-official proposal. However, this cause-effect scenario explores how this concept could be applied. It allows the Data Controller to create new consent terms related to a subset of actions that the Data Subject should know. The pluggable consent must have at most the same time rang to the original consent. Moreover, it also allows the Data Subject to revoke a specific plugged consent term and not the original; hence, it enables the Data Subject to keep sharing the data and use the services proposed by the Data Controller.

Let's consider that Maria wants to subscribe to a new service offered by Bank B. In this case, Maria has to agree with a pluggable consent sent by the bank. This consent term should clarify which service the bank is offering.

It is important to note that any missing (null) information will be replaced by the general consent term.

Figure 21 depicts the scenario of plugging a new consent term.

<div>
<img src="https://github.com/phalves/ConFIA/blob/main/img/PluggableConsentProcess.png?raw=1" width="800"/>
</div>
Fig.21 - Pluggable Consent Process.

First let's create a new consent term.

In [ ]:
?- createConsentTerm(14,'Bank B','Maria','maria@mail.com','personal_and_transactional','offer_products_and_services',
                'create_specific_offers',
                'data_analytics',
                31536000,
                'none',
                'e-mail',
                'lgpd@bankb.br',
                'SHA256',
                'Authorized employees can access the data only',
                'Bank B private cloud').

true.

In [ ]:
?- assertz(dsRight(processingConfirmation,dataSubject('Maria'),dataController('Bank B'))),
    assertz(log('Data Subject can ask if the Data Controller is processing his/her data','Explanation','Permission',1639970970)).

true.

In [ ]:
?- checkConsentTerm(id(14),dataController('Bank B'),
                        dataSubject('Maria'),
                        purpose('Bank B','offer_products_and_services'),
                        'create_specific_offers',
                        'data_analytics',
                        31536000,
                        'none',
                        'e-mail',
                        'lgpd@bankb.br',
                        1639970970).    

true.

In [ ]:
?- setThatdsAgreeWithConsentTerms(id(14),
                                dataSubject('Maria'),
                                dataController('Bank B'),
                                requestFormat('Direct','Maria','null'),
                                personalData('Maria','maria@mail.com'),
                                sensitiveData('Maria','transactional_data'),
                                startDate(1639970970),
                                endDate(EndDate)), EndDate is 1639970970+31536000.

EndDate = 1671506970 .

In [ ]:
?- setDSRights(dataSubject('Maria'),dataController('Bank B'),startDate(1639970970)).

true.

Now, let's create the pluggable consent structure.

In [ ]:
% Description: This function defines a consent term including all required information described in the LGPD Art. 9
% The consent term must inform:
%    i. Data Controller
%   ii. Data Subject
%  iii. Specific Purpose
%   iv. Form 
%    v. Time length of processing
%   vi. The purpose when sharing the data with third parties, when applied
%  vii. Communication channel to the data subject request any information
% viii. Data Controller contact
%   iv. Cryptography Algoritm
%    x. Access Policies
%   xi. Storage Platform
%  Xii. Identification to Plug this term to another consent term


createPluggableConsentTerm(ID,DC,DS,PData,SData,Purpose,
                SpecificPurpose,
                Form,
                TimeLength,
                ThirdPartyPurpose,
                Channel,
                DCContact,
                CA, AP, SP, CP) :-

                assertz(id(ID)),
                assertz(dataSubject(DS)),
                assertz(dataController(DC)),
                assertz(personalData(DS,PData)),
                assertz(sensitiveData(DS, SData)),
                assertz(purpose(DC,DS,Purpose)),
                assertz(specificPurpose(DC,DS,Purpose,SpecificPurpose)),
                assertz(form(DC,DS,Purpose,SpecificPurpose,Form)),
                assertz(timeLength(DC,DS,Purpose,SpecificPurpose,TimeLength)),
                assertz(thirdyPartySharingPurpose(DC,DS,Purpose,SpecificPurpose,
                                                    TimeLength,ThirdPartyPurpose)),
                assertz(channelToProvideInformation(DC,DS,Channel,DCContact)),
                assertz(criptographyAlgoritm(CA)),
                assertz(accessPolicies(AP)),
                assertz(storagePlatform(SP)),
                assertz(consentToPlug(CP)).

In [ ]:
% Pluggable consent params

%    i. DC: Data Controller
%   ii. DS: Data Subject
%  iii. PData: Personal Data
%   iv. SData: Sensitive Data
%    v. Purpose: Purpose of data usage
%   vi. SpecificPurpose: Specific motivation to request data
%  vii. Form: Processing method
% viii. TimeLength: How long the data will be shared
%   ix. ThirdPartyPurpose: If the data will be shared with third parties, explain why
%    x. Channel: Communication channel in case of doubts or requests related to the data
%   xi. DCContact: Contact of a person who is responsible of processing the data
%  xii. CA: Criptography algoritms applied
% xiii. AP: Access policies
%  xiv. SP: Storage platform
%   xv. CP: Consent to plug

?- createPluggableConsentTerm(15,'Bank B stocks department','Maria','null','transactional_data','offer_products_and_services',
                'offer_stock_investiments',
                'data_analytics',
                15811200,
                'none',
                'e-mail',
                'stocks@bankb.br',
                'SHA256',
                'Only the stocks department can access the data',
                'Bank B private cloud',
                11).

true.

The pluggable consent term will be valid if the consent term with the university is valid only.

In [ ]:
?- consentTermStatus(id(14),dataController('Bank B'),dataSubject('Maria'),status('Valid')),
  assertz(consentTermStatus(id(15),dataController('Bank B stocks department'),dataSubject('Maria'),status('Valid'))).

true.

The following function will check if there is valid consent. If so, the data subject will accept the plugged consent.

In [ ]:
% Description: This function sets that the Data Subject agreed with the consent term.
% This function receives the params:
%    i. Consent ID
%   ii. Data Subject
%  iii. Data Controller
%   iv. Request Format (Direct/Expresso or Proxy/Tacito)
%   iv. Personal Data
%    v. Sensitive Data
%   vi. Start Date - Timestamp
%   vi. End Date - Timestamp

setThatdsAgreeWithPluggableConsentTerm(id(ID),dataSubject(DS),
                                dataController(DC),
                                requestFormat(RF,DS,LPC),
                                personalData(DS,PData),
                                sensitiveData(DS,SData),
                                startDate(StartTS),
                                endDate(EndTS),
                                consentRelationship(id(ID),id(CP))) :-
    consentTermStatus(id(ID),dataController(DC),dataSubject(DS),status('Valid')),
    assertz(consentRelationship(id(ID),id(CP))),
    
    assertz(origin(id(ID),dataSubject(DS),dataController(DC),requestFormat(RF,DS,LPC))),
    assertz(requestFormat(RF,DS,LPC)),
    
    assertz(dsAgreeWithConsentTerms(dataSubject(DS),dataController(DC),startDate(TS),endDate(TS))),
    assertz(log('Data Subject agrees with consent term','Communication','Compliance',StartTS)),

    assertz(dcIsCollectingDSData(id(ID),dataController(DC),dataSubject(DS),personalData(DS,PData),sensitiveData(DS,SData),startDate(StartTS),endDate(EndTS))),
    assertz(log('Data Controller can collect the Data Subject information','Explanation','Permission',StartTS)),
    
    assertz(dcIsStoringDSData(id(ID),dataController(DC),dataSubject(DS),personalData(DS,PData),sensitiveData(DS,SData),startDate(StartTS),endDate(EndTS))),
    assertz(log('Data Controller can store the Data Subject information','Explanation','Permission',StartTS)),
        
    assertz(dcIsProcessingDSData(id(ID),dataController(DC),dataSubject(DS),personalData(DS,PData),sensitiveData(DS,SData),startDate(StartTS),endDate(EndTS))),
    assertz(log('Data Controller can process the Data Subject information','Explanation','Permission',StartTS)).

In [ ]:
?- setThatdsAgreeWithPluggableConsentTerm(id(15),
                                dataSubject('Maria'),
                                dataController('Bank B stocks department'),
                                requestFormat('Direct','Maria','null'),
                                personalData('Maria','maria@mail.com'),
                                sensitiveData('Maria','transactional_data'),
                                startDate(1639970970),
                                endDate(EndDate),
                                consentRelationship(id(15),id(14))), EndDate is 1639970970+15811200.

EndDate = 1655782170 .

Now, the data subject can ask to the Bank B if the data controller mentioned at the plugged consent is processing his data.

In [ ]:
% This function defines the right to request processing confirmation to the Data Subject

dsRight(processingConfirmation,dataSubject('Maria'),dataController('Bank B stocks department')).
?- assertz(log('Data Subject can ask if the Data Controller is processing his/her data','Explanation','Permission',1639970979)).

true.

In [ ]:
% This is a function call returns true if all Data Subject's right was associated to him/her.

?- setDSRights(dataSubject('Maria'),dataController('Bank B stocks department'),startDate(1639970979)).

true.

Let's check who is processing Maria's data

In [ ]:
?- dcIsCollectingDSData(id(ID),
dataController(DController),
dataSubject('Maria'),
personalData('Maria',PData),sensitiveData('Maria',SData),startDate(SDate),endDate(EDate)), ID>11.

ID = 14, DController = Bank B, PData = maria@mail.com, SData = transactional_data, SDate = 1639970970, EDate = Variable(170) ;
ID = 15, DController = Bank B stocks department, PData = maria@mail.com, SData = transactional_data, SDate = 1639970970, EDate = Variable(85) .

What will happen when the general consent is not valid anymore?

Here we can observe that there is a consent relationship between the consent 14 and 15.

In [ ]:
?- consentRelationship(id(Pluggable),id(General)).

Pluggable = 15, General = 14 .

In [ ]:
?- id(X).

X = 14 ;
X = 15 .

Now, let's revoke the general consent (id 14)

In [ ]:
% This is a function call returns true if all Data Subject's request was successfully performed.
?- setDSRevokeConsent(id(14),
                        dataSubject('Maria'),
                        dataController('Bank B'),
                        personalData('Maria','maria@mail.com'),
                        sensitiveData('Maria','transactional_data'),
                        now(1639970980),
                        startDate(1639970970),
                        endDate(EndDate)
                        ), 
                        EndDate is 1639970970+31536000.

EndDate = 1671506970 .

As we can observe, the consent id 14 does not exist anymore, but there is a pluggable consent.

In [ ]:
?- id(X).
?- consentRelationship(id(Pluggable),id(General)).

X = 15 .
Pluggable = 15, General = 14 .

Thus, the data controller should also revoke the pluggable consent. It means that the consent term status should be invalid after revocation.

In [ ]:
?- setDSRevokeConsent(id(15),
                        dataSubject('Maria'),
                        dataController('Bank B stocks department'),
                        personalData('Maria','maria@mail.com'),
                        sensitiveData('Maria','transactional_data'),
                        now(1639970990),
                        startDate(1639970970),
                        endDate(EndDate)
                        ), 
                        EndDate is 1639970970+15811200.

EndDate = 1655782170 .

In [ ]:
?- consentTermStatus(id(ID),dataController(DataController),dataSubject('Maria'),status(Status)).

ID = 14, DataController = Bank B, Status = Invalid ;
ID = 15, DataController = Bank B stocks department, Status = Invalid .

In [ ]:
?- id(X).

false.

In this sense, the red entities are affected by the pluggable consent, as depicted in Figure 22.

As now there is a new data controller, the access restriction, sharing policies, and his identification might be different from the general consent. This new pluggable consent may also change the time range of this consent term.

This pluggable consent changes the access right; now, if the data subject asks this new data controller if the data controller is collecting and processing his data, he will have a positive message. 

Moreover, the purpose limitation will be different from the general consent term, and, like any other consent term, the data subject can request consent revocation anytime.

However, as depicted on Anonymization Scenario, the changes in the consent term may affect some data subjects' actions. For instance, if a student revokes a discipline consent term, this student might not be allowed to participate in the class activities.

![PluggableConsentImpact](./img/PluggableConsentImpact.png "Pluggable Consent Impact")
Fig.22 - Pluggable Consent Impact.

In [ ]:
?- log(Event,Type,DeonticOperator,Date), Date >= 1639970970 {-1}.

Event = Data Subject can ask if the Data Controller is processing his/her data, Type = Explanation, DeonticOperator = Permission, Date = 1639970970 ;
Event = Data Subject verified the consent term and it was ok, Type = Explanation, DeonticOperator = Obligation, Date = 1639970970 ;
Event = Data Subject agrees with consent term, Type = Communication, DeonticOperator = Compliance, Date = 1639970970 ;
Event = Data Controller can collect the Data Subject information, Type = Explanation, DeonticOperator = Permission, Date = 1639970970 ;
Event = Data Controller can store the Data Subject information, Type = Explanation, DeonticOperator = Permission, Date = 1639970970 ;
Event = Data Controller can process the Data Subject information, Type = Explanation, DeonticOperator = Permission, Date = 1639970970 ;
Event = Data Subject can now have all foressen rights, Type = Explanation, DeonticOperator = Permission, Date = 1639970970 ;
Event = Data Subject agrees with consent term, Type = Communication,